En este ejercicio nos queda por limpiar las columnas de age y species, así que manos a la obra. Vamos a ver que podemos hacer para cada una de las columnas y os dejaremos algunas pistas 😉.

In [1]:
import pandas as pd
import numpy as np
import sidetable
import re

In [2]:
df=pd.read_csv('../EDA/datos/attack_limpieza1.csv', index_col=0)

In [3]:
df.head(2)

,case_number,year,type,country,activity,age,time,species_,mes,fatal_,sex2
0,1800.00.00,1800,Unprovoked,seychelles,a corsair's boat was overturned,NaN,NaN,NaN,NaN,y,F
1,1797.05.28.R,1797,Unprovoked,NaN,Dropped overboard,NaN,NaN,NaN,May,y,NaN


1. Columna de species: 
   
   Si exploramos esta columna en detalle nos podemos dar cuenta que tenemos muchos valores únicos y esto hace que sea muy difícil trabajar con esta columna. Lo que vamos a hacer es clasificar los tiburones en 5 especies diferentes, las más comunes, que incluyen el tiburón blanco (White), el tiburón tigre (Tiger ), el tiburón gris (Grey), el tiburón limón (Lemon) y el tiburón toro (Bull). El resto de las especies las incluiremos en un único grupo que podremos llamar "Unspecified". Ahora nos podemos sentir un poco abrumadas y no saber como enfrentarnos a este reto, pero don't worry, os dejamos por aquí unas 💡 pistas 💡 para que os ayuden a entender cómo hacerlo.
   - Los valores de las columnas son strings por lo que podremos usar regex para buscar palabras clave en cada celda y asignarlo a una de las categorías
    que hemos definido previamente.
         
        *imaginemos que el valor de una celda es el siguiente

                    'White shark, 3.5 m'

        *tendremos que buscar el patrón de regex que nos permita extraer White shark de ese string y que nos devuelva solo White shark. 
        Un patrón que podriamos usar es: 

                    patron_blanco = r".*[Ww](hite|HITE).*" # esto podría ser así porque puede estar en mayúsculas o en minúsculas. 

        *de la misma forma que hemos sacado el patron para el tiburón blanco, tendremos que sacar los patrones para las otras 4 especies que queremos "encontrar". 

    - Tendremos que crearnos una función que aplicaremos sobre nuestra columna species_ para que nos devuelva una nueva columna con los valores clasificados en función de los patrones de regex que hayamos definido.

In [4]:
patronblanco= '.*([W|w]hite|HITE).*'

patrontigre= '.*([T|t]iger|IGER).*'

patronbull= '.*([B|b]ull|ULL).*'

patrongrey= '.*([G|g]rey|REY).*'

patronlemon= '.*([L|l]emon|LEMON).*'


In [5]:
df["species_"].fillna(value="NAN", axis=0, inplace=True)

In [6]:
#Pair Lucia Lara Anais 
def tiburon(val):

    if re.findall(patronblanco,val):
        df['cleaned_species']= 'White'
        return val.replace(val,'White')
    elif re.findall(patrontigre,val):
        df['cleaned_species']= 'Tiger'
        return val.replace(val,'Tiger')  
    elif re.findall(patronbull,val):
        df['cleaned_species']= 'Bull'
        return val.replace(val,'Bull')
    elif re.findall(patrongrey,val):
        df['cleaned_species']= 'Grey'
        return val.replace(val,'Grey')
    elif re.findall(patronlemon,val):
        df['cleaned_species']= 'Lemon'
        return val.replace(val,'Lemon')
    else:
         return val.replace(val,'Undefined') 
    

In [7]:
df['cleaned_species']=df['species_'].apply(tiburon)

In [8]:
df.sample(10)

,case_number,year,type,country,activity,age,time,species_,mes,fatal_,sex2,cleaned_species
2120,2001.04.12.c,2001,Unprovoked,usa,Surfing,21,16h05,Possibly a juvenile blacktip or spinner shark,Apr,n,M,Undefined
3789,1965.04.10,1965,Unprovoked,usa,Spearfishing,28,11h00,"White shark, 10' to 12'",Apr,n,M,White
4044,1961.08.01,1961,Provoked,papua new guinea,"Fishing, two large sharks passed. He speared o...",NaN,NaN,NAN,Aug,n,M,Undefined
1818,2004.07.19.R,2004,Boating,australia,Fishing,NaN,NaN,4 m [13'] white shark,Jul,n,NaN,White
1855,2004.03.06,2004,Unprovoked,australia,Spearfishing,47,NaN,"Bronze whaler shark, 1.5 m [5']",Mar,n,M,Undefined
3832,1964.07.15,1964,Invalid,usa,"Yacht Gooney Bird foundered, 4 survivors on raft",NaN,NaN,Invalid,Jul,NaN,NaN,Undefined
2109,2001.05.18,2001,Provoked,philippines,Shark fishing,24,NaN,NAN,May,n,M,Undefined
4353,1957.07.09,1957,Unprovoked,australia,Diving for trochus,36,NaN,9' shark,Jul,n,M,Undefined
1279,2009.07.05,2009,Unprovoked,usa,Boogie Boarding,12,16h00,NAN,Jul,n,F,Undefined
3107,1983.04.02.a,1983,Invalid,south africa,NaN,NaN,NaN,Questionable incident,Apr,NaN,M,Undefined


2. Columna de age: 
   
   es una columna de tipo string pero debería ser de tipo integer. Además, en esta columna nos vamos a encontrar con algunos errores tipográficos, estos incluyen:
   - Edad en formato string
   - Edades separadas por &, or, to, >
   - Edades con ?
   Vamos con algunas 💡 pistas 💡 para que os ayuden a entender cómo hacerlo:
   - Primero tendremos que eliminar todos esos símbolos especiales que nos aparecen. De nuevo, podremos usar regex para extraer unicamente los números que es lo que nos interesa. Usar este regex en una función para sacar solo los números.
   - Puede que os salga un error similar a este:
  
                     TypeError: expected string or bytes-like object
                  
     Para solucionar este problema, antes de nada tendréis que ejecutar este código para que no os de error:

                     df['nombre_columna'] = df['nombre_columna'].astype(str)
                     
   - Una vez que hayáis extraido los números, os daréis cuenta que hay celdas que tienen más de una edad. Tendréis que decidir que hacer en esos casos. Os dejamos por aquí una posible opción usando un método de Pandas que os puede resultar super útil. 
   - Por último cambiad el tipo de la columna de string a integer.
  


In [9]:
df['age']=df['age'].astype(str)


In [10]:
edades=list(df['age'].unique())
print(edades)


['nan', 'young', '14', '19', '20', '15', '21', '27', '36', '25', '6', '16', '50', '13 or 14', '57', '11', '48', '18', '52', '12', '32', '10', '34', '30', '60', '33', '29', '54', '41', '37', '56', '69', '38', '55', '35', '46', '45', '40s', '28', '24', '26', '49', '22', '7', '31', '17', '40', '13', '42', '3', '8', '82', '73', '20s', '68', '51', '39', '58', 'Teen', '47', '61', '65', '66', '43', '60s', '9', '72', '59', '64', '23', '71', '44', '62', '63', '70', '18 months', '53', '30s', '50s', 'teen', '77', '74', '28 & 26', '5', '86', '18 or 20', '12 or 13', '46 & 34', '28, 23 & 30', 'Teens', '36 & 26', '8 or 10', '84', '\xa0 ', ' ', '30 or 36', '6½', '21 & ?', '75', '33 or 37', 'mid-30s', '23 & 20', ' 30', '7      &    31', ' 28', '20?', "60's", '32 & 30', '16 to 18', '87', '67', 'Elderly', 'mid-20s', 'Ca. 33', '74 ', '45 ', '21 or 26', '20 ', '>50', '18 to 22', 'adult', '9 & 12', '? & 19', '9 months', '25 to 35', '23 & 26', '1', '(adult)', '33 & 37', '25 or 28', '37, 67, 35, 27,  ? & 27',

In [11]:
def edad(x):
    try:
        datos=re.findall('\d+',x)
        return datos
    except:
        pass

In [12]:
df['age']=df['age'].apply(edad)

In [13]:
df.sample(10)

,case_number,year,type,country,activity,age,time,species_,mes,fatal_,sex2,cleaned_species
3912,1963.04.20,1963,Unprovoked,usa,Swimming,[25],13h50,"Hand found in gut of 2.9 m to 3.3 m [9'7"" to 1...",Apr,y,M,Undefined
2682,1992.11.29,1992,Unprovoked,usa,Spearfishing / free diving,[31],15h30,"Mako shark, 1.8 m [6']",Nov,n,M,Undefined
5592,1908.05.13,1908,Sea Disaster,indonesia,native boats sunk in storm,[],NaN,NAN,May,y,F,Undefined
3149,1982.02.27,1982,Unprovoked,usa,Surfing,[18],NaN,4' to 5' shark,Feb,n,M,Undefined
356,2017.01.09,2017,Unprovoked,indonesia,Surfing,[48],07h30,"Bull shark, 1 m",Jan,n,M,Bull
2132,2001.03.23,2001,Unprovoked,usa,Body-boarding,[],14h40,NAN,Mar,n,M,Undefined
952,2012.06.10,2012,Provoked,italy,Attempting to rescue an injured & beached shark,[57],Morning,"Blue shark, 2.5m",Jun,NaN,M,Undefined
3258,1979.05.00.c,1979,Unprovoked,new caledonia,Fishing,[],NaN,NAN,NaN,n,NaN,Undefined
5421,1919.11.18,1919,Unprovoked,australia,Sitting in a boat,[],NaN,NAN,Nov,n,M,Undefined
5308,1926.05.18,1926,Unprovoked,usa,Swimming,[],NaN,"White shark, 3.8 m [12.5']",May,y,M,White


In [14]:
df=df.explode(["age"], ignore_index=False)

In [15]:
df.sample(10)

,case_number,year,type,country,activity,age,time,species_,mes,fatal_,sex2,cleaned_species
387,2016.09.15.R,2016,Boating,australia,Fishing,NaN,NaN,NAN,Sep,n,NaN,Undefined
519,2015.10.07,2015,Unprovoked,australia,Surfing,18,16h50,NAN,Oct,n,M,Undefined
456,2016.04.13,2016,Unprovoked,usa,Spearfishing,26,NaN,"Bull shark, 7'",Apr,n,M,Bull
1224,2009.12.29,2009,Unprovoked,south africa,Wading,20,08h30,NAN,Dec,n,M,Undefined
5051,1935.12.23,1935,Provoked,australia,Cruising,NaN,NaN,4.3 m [14'] shark,Dec,n,NaN,Undefined
4455,1955.04.12,1955,Unprovoked,papua new guinea,Spearfishing,NaN,NaN,6' shark,Apr,n,M,Undefined
1796,2004.10.09.a,2004,Unprovoked,south africa,Surfing,34,Morning,"Raggedtooth shark, 1.5 to 2 m [5' to 6.75']",Oct,n,M,Undefined
582,2015.06.14.a,2015,Unprovoked,usa,Wading,12,16h12,Bull shark,Jun,n,F,Bull
4297,1958.10.00.a,1958,Unprovoked,papua new guinea,Collecting shells,26,NaN,NAN,Oct,n,M,Undefined
3292,1978.00.00.a,1978,Unprovoked,vanuatu,Swimming,NaN,NaN,NAN,NaN,y,F,Undefined


In [16]:
df.drop_duplicates(inplace=True)

In [17]:
df.describe()

,year
count,6327.000000
mean,1927.127865
std,281.576548
min,0.000000
25%,1942.000000
50%,1976.000000
75%,2005.000000
max,2018.000000


In [18]:
df["age"]=df["age"].astype(float)

In [19]:
media=round(np.mean(df["age"]))
media

27

In [20]:
#Decidimos reemplazar los nulos por la media de la columna 
df["age"]=df["age"].fillna(media)

In [21]:
df["age"]=df["age"].astype(int)

In [22]:
df.dtypes

case_number        object
year                int64
type               object
country            object
activity           object
age                 int64
time               object
species_           object
mes                object
fatal_             object
sex2               object
cleaned_species    object
dtype: object

3. Guarda el csv con las columnas limpias para seguir trabajando con este dataframe limpio.

In [24]:
df.to_csv("../EDA/datos/attack_limpieza2.csv")